In [1]:
import secrets
secret_key = secrets.token_hex(32)
print(secret_key)

d6d2e8a428b9ea5e7856e66f736dd74de2c8979fe38daa00dc3a9280c116a3e8


In [1]:
import psycopg2

In [2]:
try:
    db_conn=psycopg2.connect(
        dbname="BlueBash",
        user="postgres",
        password="7895",
        host="127.0.0.1",
        port="5432"
    )
    print("Connection to PostgreSQL successful!")
    
    # Now you can execute SQL queries or perform database operations using this connection
    
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)


Connection to PostgreSQL successful!


In [3]:
db_cursor=db_conn.cursor()

In [4]:
db_cursor.execute("""
    CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    username VARCHAR(100) UNIQUE,
    password VARCHAR(100)
);
    """)

db_conn.commit()
db_conn.close()

In [5]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Annotated

In [6]:
app=FastAPI()

In [7]:
class UpdateRequest(BaseModel):
    id: int
    username: str
    password: str
    name: str


In [8]:
@app.get('/')
def read_root(data):
    print(data)
    return {"Welcome": "Gulam Sarvar"}

In [11]:
@app.put("/update_table")
async def update_table(update_request: UpdateRequest):
    try:
        update_query = """
        UPDATE your_table_name
        SET username = %s, password = %s, name = %s
        WHERE id = %s;
        """
        # Execute the SQL query to update the record
        db_cursor.execute(update_query, (update_request.username, update_request.password, update_request.name, update_request.id))
        # Commit the transaction
        db_conn.commit()
        return {"message": "Record updated successfully."}
    except (Exception, psycopg2.Error) as error:
        # Rollback the transaction in case of error
        if db_conn:
            db_conn.rollback()
        raise HTTPException(status_code=500, detail=f"Error updating record: {error}")
    finally:
        # Close the database connection
        if db_conn:
            db_cursor.close()
            db_conn.close()